In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options 
import time
from selenium.webdriver.support.ui import Select
import pandas as pd
import numpy as np

# Academic Ranking of World Universities
# Shanghai Ranking

## Web scraping dos dados do ranking Shanghai

Para o web scraing dos dados será usada a biblioteca selenium

In [ ]:
#Necessário caso ChromeDriver não esteja no path
'''
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
'''

### Scrap de rankings individuais

In [ ]:
ano='2008' #especificar o ano do ranking 
website='https://www.shanghairanking.com/rankings/arwu/'+ano
driver = webdriver.Chrome()
driver.get(website) #abre a pagina
driver.maximize_window()  #maximizar a página

Na primeira coluna da tabela estão os seguintes headers:

- World Rank;
- Institution;
- Country/Region;
- National/Regional Rank;
- Total Score;
- Menu com as feutures:
    - Alumni;
    - Award;
    - HiCi;
    - N&S;
    - PUB;
    - PCP.

As features estão em um menu dropdown. Portanto durante o processo de scraping deverá ocorrer interação com esse menu para que ser possível coletar os dados para cada uma delas. Além disso, os dados para todas as universidades estão 
em diversas páginas, sendo assim será necessário interagir com a numeração das páginas para coletar os dados do completos do ranking


In [ ]:
features=["Alumni",'Award','HiCi','N&S','PUB','PCP'] #features calculadas para cada universidade. O significado de cada uma dela pode ser encontrado no notebook com a análise dos dados dos rankings
#Essas features estão em um menu dropdown na página

#Dicionário que receberá os dados
ranking={'World Rank' :[],
              'Institution':[],
              'Country'    :[],
              'Regional Rank':[],
              'Total Score':[],
                'Alumni': [],
                'Award' : [],
                'HiCi'  : [],
                'N&S'  : [],
                'PUB'   : [],
                'PCP'   : []}


 

In [ ]:
#Path onde serão salvos os dados
path_to_save='/home/rodolfo/Insync/rodolfopcruz2@gmail.com/Google Drive/Estudo/Python_Projects/education_data/dados/Shanghai World University Rankings/web_scraping/'

In [ ]:
#número de páginas
#os ranking estão divididos em diversas páginas
#em geral em cada página estão os dados para 30 universidades
pagination=driver.find_element('xpath',"//ul[contains(@class,'pagination')]")
paginas=pagination.find_elements('xpath','./li') #botões com as numerações das páginas
num_paginas=paginas[-2].text #o promeiro botão é um seta para recuar e o último uma seta para avançar. Entre esses dois estão os botões com os números das páginas
num_paginas #total de pádinas onde estão contidos os dados

In [ ]:
#web sraping dos dados do ranking do ano selecionado

botao_next=pagination.find_element('xpath','//li[@class=" ant-pagination-next"]') #Botaão para avançar a página
time.sleep(3)

pagina_atual=1 #primeira página
while pagina_atual<=int(num_paginas): #varrer todas as páginas com os dados do ranking

    botoes_header=driver.find_elements(by='xpath',value='//th//img') #localizar botão dropdown. Ele está nos headers da tabela. Ao clicar nesse botão o menu dropdow é aberto
    botoes_header=botoes_header[-1] #nos headers estão dois botões dropdown. O primeiro seleciona o país e o segundo a feature. O botão desejado é o segundo.
        
    opcoes=driver.find_elements(by='xpath',value='//th') #localizar as opções contidas nos menus dropdowns no header da tabela
    opcoes=opcoes[-1] #segundo menu dropdown no header da tabela
    opcoes=opcoes.find_elements('xpath','.//li') #pegar as opções contidas no menu dropdown. Opcoes é uma lista em que cada elemento corresponde a uma feature
        
    table=driver.find_element(by='xpath',value="//tbody") #dados na tabela sem o header (somente a tabela com os dados, sem os nomes das colunas)
    table_rows=table.find_elements(by='xpath',value="./tr") #rows da tabela

    first_feature=True #indicar se é a primeira feature do meno dropdown.  tabela na página tem 6 colunas. Na sexta está um menui dropdrown para seleção da feature desejada.
                       #Alterando a feature no menu dropdown nao tem nenhum efeito nas 5 colunas anteriores. Então se first_feature=True será feito o scrap dos dados das seis colunas.Caso 
                       #contrário somente da última coluna, uma vez que os dados das demais colunas já foram coletados
    
    for feature in features:  #selecionar uma feature     
        botoes_header.click() #clicar no botão para abrir o menu dropdown
        time.sleep(1)
        for opcao in opcoes: #cada opcao é uma das features no menu dropdown
            if opcao.text==feature: #opcao.text retorna o nome da feature. Seleciona a opção desejada da feature no menu dropdown
                opcao.click() #clica na opcao
                if first_feature: #se true fará scrap de seis colunas
                     for row in table_rows: #loop por todas as linhas da tabela
                        td=row.find_elements(by='xpath',value='./td') #td são as colunas da tabela.
                        ranking['World Rank'].append(td[0].text)#primeira coluna é a posicao no ranking mundial
                        ranking['Institution'].append(td[1].text)#primeira coluna é a posicao no ranking mundial
                        codigo_pais=td[2].find_element(by='xpath',value='div[@class="region-img"]') #na coluna com o pais está uma url para a bandeira do pais
                        ranking['Country'].append(codigo_pais.get_attribute("style").split('/')[-1].split('.')[0]) #extrair o nome do pais da url
                        ranking['Regional Rank'].append(td[3].text) #terceria coluna é a posição da universidade no ranking do próprio país
                        ranking['Total Score'].append(td[4].text)   #pontuação final obtida pela universidade
                        ranking[feature].append(td[5].text)         #primeira das 6 fatures no menu dropdown
                        first_feature=False #first_feature=False para que na próxima iteração sejam coletados somente os dados da última coluna                    
                else:
                    for row in table_rows: #loop por todas as linhas da tabela
                        td=row.find_elements(by='xpath',value='./td') #td são as colunas da tabela.
                        ranking[feature].append(td[5].text) #coletar os dados das features no menu dropdown                
                time.sleep(5)
                break
    print(pagina_atual)
    time.sleep(3)  
    botao_next.click() #clicar no botão para avançar a página
    #print('')
    time.sleep(3)
    botao_top=driver.find_element('xpath','//div[@class="back-top"]') #botão para voltar ao topo da página
    botao_top.click() #clicar no botão para voltar ao topo da página
    time.sleep(10)
    pagina_atual+=1
ranking=ranking=pd.DataFrame.from_dict(ranking) #converter o dicionário para dataframe
ranking.to_csv(path_to_save+'ranking_'+ano+'.csv',index=False) #salvar o dataframe no diretório especificado. 
driver.quit() #fechar a página
              


# 2004 a 2023

Automatizar o scraping dos rankings, de forma que os ranking para diversos anos sejam coletados de uma única vez.
Os ranking para cada ano serão salvos em arquivos diferentes, no seguinte formato:

- ranking_20xx.csv

In [ ]:
anos=[str(i) for i in range(2006,2024)] #lista para definir os rankings que serão coletados
for ano in anos: 
    website='https://www.shanghairanking.com/rankings/arwu/'+ano #abrir a pagina do ano selecionado
    driver = webdriver.Chrome()
    driver.get(website) #abre a pagina
    driver.maximize_window()  #maximizar a página 
    time.sleep(20) #aguardar o carregamento da página


    #Dicionário que receberá os dados
    ranking={'World Rank' :[],
              'Institution':[],
              'Country'    :[],
              'Regional Rank':[],
              'Total Score':[],
                'Alumni': [],
                'Award' : [],
                'HiCi'  : [],
                'N&S'  : [],
                'PUB'   : [],
                'PCP'   : []}
    
    #verificar o número de páginas. Total de páginas para cada ranking
    pagination=driver.find_element('xpath',"//ul[contains(@class,'pagination')]")
    paginas=pagination.find_elements('xpath','./li')
    num_paginas=paginas[-2].text # a numeraçãio da página está em diversos botões na parte inferior da página. O primeiro botão é um seta de recuo (<), que indica
                                 # retornar uma página. O último botão é um seta de avanço (>) para avançar uma página. Entre esse dois estão os botões com os 
                                 #números específicos de cada página
    num_paginas #total de páginas de um ranking
    botao_next=pagination.find_element('xpath','//li[@class=" ant-pagination-next"]') #botão para avançar a página
    time.sleep(3)

    pagina_atual=1 #primera página do ranking
    while pagina_atual<=int(num_paginas): #varrer todas as páginas do ranking
        
        botoes_header=driver.find_elements(by='xpath',value='//th//img') #localizar botão dropdown para seleção das features. Ele está nos headers da tabela. 
                                                                        #Ao clicar nesse botão o menu dropdow é aberto
        botoes_header=botoes_header[-1] #nos headers estão dois botões dropdown. O primeiro seleciona o país e o segundo a feature. O botão desejado é o segundo.
                                        #botões_header é o botão que deve ser clicado para abertura do menu dropdown
        opcoes=driver.find_elements(by='xpath',value='//th') #menus dropdown no header da tabela. opcoes são as opções contidas no menu dropdown
        opcoes=opcoes[-1] #segundo menu dropdown no header da tabela
        opcoes=opcoes.find_elements('xpath','.//li') #pegar as opções contidas no menu dropdown. Opcoes é uma lista em que cada elemento corresponde a uma feature
        
        table=driver.find_element(by='xpath',value="//tbody") #dados na tabela sem os headers, somente os dados das colunas sem os nomes de cada coluna
        table_rows=table.find_elements(by='xpath',value="./tr") #rows da tabela

        '''
        first_feature é usada para determinar quais colunas da tabela serão coletadas. A tabela tem várias colunas, somente em uma delas a feature na coluna pode ser selecionada.
        Quando a feature na coluna é selecionada, somente os dados da coluna são alterados. Então quando first_feature==true os dados de todas as colunas serão coletados, caso contrário
        serão coletados somentes os dados da coluna com a feature selecionada
        '''
        first_feature=True 
        for feature in features:       
          botoes_header.click() #clicar no botão para abrir o menu dropdown
          time.sleep(5)
          for opcao in opcoes: #cada opcao é uma das features no menu dropdown
            if opcao.text==feature: #opcao.text retorna o nome da feature. Seleciona a opção desejada da feature
              opcao.click()
              if first_feature: #coletar dados de todas as colunas na tabela
                for row in table_rows: #loop por todas as linhas da tabela
                  td=row.find_elements(by='xpath',value='./td') #td são as colunas da tabela.
                  ranking['World Rank'].append(td[0].text)#primeira coluna é a posicao no ranking mundial
                  ranking['Institution'].append(td[1].text)#primeira coluna é a posicao no ranking mundial
                  codigo_pais=td[2].find_element(by='xpath',value='div[@class="region-img"]') #no pais está url de uma imagem da bandeira do pais
                  ranking['Country'].append(codigo_pais.get_attribute("style").split('/')[-1].split('.')[0]) #extrair o nome do pais da url
                  ranking['Regional Rank'].append(td[3].text) #posição do pais no ranking considerando somente universidades do próprio pais
                  ranking['Total Score'].append(td[4].text) #pontuação geral, considerando todos os fatores, atribuída a cada universidade
                  ranking[feature].append(td[5].text) #primeira das features no menu dropdown
                  first_feature=False #na próxima interação somente serão coletados os dados da feature selecionada no menu dropdown. As demais colunas da tabela já foram coletados
              else:
                for row in table_rows: #loop por todas as linhas da tabela
                  td=row.find_elements(by='xpath',value='./td') #td são as colunas da tabela.
                  ranking[feature].append(td[5].text)  #coletar dados das colunas restantes
              time.sleep(5)
              break
        time.sleep(2)
        botao_next.click() #avançar pagina
        time.sleep(2)
        botao_top=driver.find_element('xpath','//div[@class="back-top"]') #botão para voltar ao topo da página
        botao_top.click() #clicar no botão para voltar ao topo da página
        time.sleep(5)     
        print('Página atual: {}'.format(pagina_atual))
        print('Ano: {}'.format(ano))
        print('')
        pagina_atual+=1
    ranking=pd.DataFrame.from_dict(ranking) #converter o dicionário para pandas dataframe
    ranking.to_csv(path_to_save+'ranking_'+ano+'.csv',index=False)
    driver.quit()  
        
        
    



 